In [ ]:
import numpy as np
import sys
import os

In [ ]:
from swarmcraft.core.loss_functions import create_landscape, RastriginLandscape, EcologicalLandscape, QuadraticLandscape
from swarmcraft.core.pso import PSO
from swarmcraft.core.landscape_visualizer import LandscapeVisualizer, quick_visualize


## 🧪 Phase 4: Algorithm Tuning & Fast Convergence Test

Here, we test the PSO algorithm on a simple, convex `QuadraticLandscape`. The goal is to find parameters that lead to reliable and fast convergence (~5-10 steps), which is ideal for live demonstrations and quick testing cycles.

In [ ]:
quad_landscape = create_landscape("quadratic", dimensions=2)
fig = quick_visualize("quadratic", plot_type="2d", show_grid=True, grid_size=20)
fig.show()

In [ ]:
# PSO parameters tuned for fast convergence on a simple landscape
pso_fast_converge = PSO(
    dimensions=2,
    bounds=quad_landscape.metadata.recommended_bounds,
    loss_function=quad_landscape.evaluate,
    population_size=10,  # Smaller population is fine for simple problems
    inertia_weight=0.7,      # Lower inertia to reduce overshooting
    cognitive_coefficient=1.5, # Balance between personal and social influence
    social_coefficient=1.5,
    exploration_probability=0.05, # Very little exploration needed
    random_seed=42
)

print(f"FAST CONVERGENCE TEST")
print(f"Initial Best Fitness: {pso_fast_converge.swarm_state.global_best_fitness:.4f}")

for i in range(1, 11):
    pso_fast_converge.step()
    print(f"Step {i}: Best Fitness = {pso_fast_converge.swarm_state.global_best_fitness:.4f}")

print(f"\n✅ Convergence achieved in ~10 steps.")

---

In [ ]:
fig = quick_visualize("rastrigin", plot_type="discrete", grid_size=25, A=10.0)
fig.show()

In [ ]:
fig = quick_visualize("rastrigin", plot_type="2d", A=25.0, dimensions=2, show_grid=True)
fig.show()

In [ ]:
fig = quick_visualize("ecological", plot_type="3d")
fig.show()

In [ ]:
def simple_quadratic(x):
    """Simple quadratic function with minimum at origin"""
    return np.sum(x**2)

In [ ]:
pso = PSO(
    dimensions=2,
    bounds=[(-5, 5), (-5, 5)],
    loss_function=simple_quadratic,
    population_size=20,
    random_seed=42
)

In [ ]:
print(f"Initial best fitness: {pso.swarm_state.global_best_fitness:.3f}")
print(f"Initial best position: {pso.swarm_state.global_best_position}")

In [ ]:
for i in range(20):
    pso.step()
    if i % 5 == 0:
        print(f"Step {i}: Best fitness = {pso.swarm_state.global_best_fitness:.3f}")

print(f"Final best fitness: {pso.swarm_state.global_best_fitness:.3f}")
print(f"Final best position: {pso.swarm_state.global_best_position}")
print("✅ PSO optimization working!")

In [ ]:
rastrigin = create_landscape("rastrigin", A=10.0, dimensions=2)
print(f"Landscape: {rastrigin.metadata.name}")
print(f"Global minimum: {rastrigin.metadata.global_minimum}")
print(f"Expected minimum value: {rastrigin.metadata.global_minimum_value}")

In [ ]:
pso_rastrigin = PSO(
    dimensions=2,
    bounds=rastrigin.metadata.recommended_bounds,
    loss_function=rastrigin.evaluate,
    population_size=25,  # Larger swarm for harder problem
    exploration_probability=0.15,  # More exploration
    random_seed=42
)

In [ ]:
fitness_history = []
for i in range(50):
    pso_rastrigin.step()
    fitness_history.append(pso_rastrigin.swarm_state.global_best_fitness)

    if i % 10 == 0:
        stats = pso_rastrigin.get_pso_statistics()
        print(f"Step {i}: Fitness = {stats['global_best_fitness']:.3f}, "
              f"Explorers = {stats['exploration_stats']['explorers']}, "
              f"Diversity = {stats['diversity']:.3f}")

print(f"Final best fitness: {pso_rastrigin.swarm_state.global_best_fitness:.3f}")
print(f"Final best position: {pso_rastrigin.swarm_state.global_best_position}")


In [ ]:
visualizer = LandscapeVisualizer()

In [ ]:
particle_positions = [p.position for p in pso_rastrigin.swarm_state.particles]
particle_fitnesses = [p.fitness for p in pso_rastrigin.swarm_state.particles]
particle_ids = [p.id for p in pso_rastrigin.swarm_state.particles]

In [ ]:
fig = visualizer.plot_swarm_on_landscape(
    rastrigin,
    particle_positions,
    particle_fitnesses,
    particle_ids,
    resolution=60
)
fig.show()

In [ ]:
ecological = create_landscape("ecological")
print(f"Landscape: {ecological.metadata.name}")
print(f"Story context: {ecological.metadata.story_context[:100]}...")

In [ ]:
pso_eco = PSO(
    dimensions=2,
    bounds=ecological.metadata.recommended_bounds,
    loss_function=ecological.evaluate,
    population_size=100,
    exploration_probability=0.2,
    random_seed=42
)
print(f"Initial best cost: {pso_eco.swarm_state.global_best_fitness:.3f}")

In [ ]:
for i in range(50):
    pso_eco.step()
    if i % 5 == 0:
        position = pso_eco.swarm_state.global_best_position
        fitness = pso_eco.swarm_state.global_best_fitness
        description = ecological.describe_position(np.array(position))
        print(f"Step {i}: {description}, fitness = {fitness:.3f}")

In [ ]:
fig_eco = visualizer.plot_swarm_on_landscape(
    ecological,
    [p.position for p in pso_eco.swarm_state.particles],
    [p.fitness for p in pso_eco.swarm_state.particles],
    resolution=50
)
fig_eco.show()

In [ ]:
print("👥 Testing human interaction features...")

# Test particle information retrieval
particle_id = pso_eco.swarm_state.particles[0].id
particle_info = pso_eco.get_particle_info(particle_id)
print(f"Particle {particle_id} info:")
for key, value in particle_info.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
suggestion = pso_eco.suggest_next_position(particle_id)
print(f"Suggested next position for {particle_id}: {suggestion}")

In [ ]:
print("\n🔄 Testing forced exploration phase...")
initial_explorers = sum(1 for p in pso_eco.swarm_state.particles if p.state.value == "exploring")
print(f"Initial explorers: {initial_explorers}")

In [ ]:
test_positions = [
    [0.0, 0.0],     # Global minimum
    [1.0, 1.0],     # Decent position
    [3.0, 3.0],     # Poor position
    [-4.0, 4.0],    # Very poor position
]

In [ ]:
for pos in test_positions:
    fitness = rastrigin.evaluate(np.array(pos))
    color = rastrigin.get_fitness_color(fitness)
    frequency = rastrigin.get_fitness_audio_frequency(fitness)
    description = rastrigin.describe_position(np.array(pos))

    print(f"Position {pos}: Fitness={fitness:.3f}, Color={color}, Freq={frequency:.1f}Hz")
    print(f"  Description: {description}")

In [ ]:
print("\nEcological landscape feedback:")
for pos in test_positions:
    # Adjust positions to ecological bounds
    eco_pos = [min(10, max(0, pos[0] + 5)), min(10, max(0, pos[1] + 5))]
    fitness = ecological.evaluate(np.array(eco_pos))
    color = ecological.get_fitness_color(fitness)
    description = ecological.describe_position(np.array(eco_pos))

    print(f"Position {eco_pos}: Cost={fitness:.3f}, Color={color}")
    print(f"  Description: {description}")

In [ ]:
print("📈 Analyzing performance and statistics...")

stats = pso_rastrigin.get_pso_statistics()
print("PSO Statistics:")
for key, value in stats.items():
    if isinstance(value, dict):
        print(f"{key}:")
        for subkey, subvalue in value.items():
            if isinstance(subvalue, float):
                print(f"  {subkey}: {subvalue:.3f}")
            else:
                print(f"  {subkey}: {subvalue}")
    elif isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

In [ ]:
landscape_stats = visualizer.analyze_landscape_statistics(rastrigin, resolution=100)
print(f"\nRastrigin Landscape Analysis:")
for key, value in landscape_stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

In [ ]:
dashboard_fig = visualizer.create_dashboard(ecological)
dashboard_fig.show()

In [ ]:
dashboard_fig = visualizer.create_dashboard(rastrigin)
dashboard_fig.show()

In [ ]:
print("=" * 60)
print("🏆 SWARMCRAFT TESTING SUMMARY")
print("=" * 60)
print("✅ Landscape creation and evaluation")
print("✅ PSO optimization with automatic initialization")
print("✅ Interactive Plotly visualizations")
print("✅ Human interaction features (particle info, suggestions)")
print("✅ Exploration/exploitation phase control")
print("✅ Color and audio feedback generation")
print("✅ Statistical analysis and dashboards")
print("✅ Error handling and edge cases")
print("=" * 60)
print("🚀 Ready for FastAPI backend and Svelte frontend!")
print("🎯 Next: Redis + WebSockets + Session management")
print("=" * 60)